БД: `PostgreSQL` \
Описание таблиц БД `test`:

1. Таблица `product_mag`:
 
- `product_id` (INTEGER): Уникальный идентификатор продукта (первичный ключ).
- `name` (VARCHAR(255)): Название продукта.
- `group1` (VARCHAR(255)): Первая группа, к которой принадлежит продукт.
- `group2` (VARCHAR(255)): Вторая группа, к которой принадлежит продукт.
- `group3` (VARCHAR(255)): Третья группа, к которой принадлежит продукт.
- `weight` (DECIMAL(10, 2)): Вес продукта в килограммах.
- `shelf_life` (INTEGER): Срок годности продукта в месяцах.

2. Таблица `warehouses_mag`:
- `warehouse_id` (INTEGER): Уникальный идентификатор склада (первичный ключ).
- `name` (VARCHAR(255)): Название склада.
- `city` (VARCHAR(255)): Город, в котором расположен склад.
- `date_open` (DATE): Дата открытия склада.
- `date_close` (DATE): Дата закрытия склада (если имеется).

3. Таблица `orders_mag`:
- `order_id` (INTEGER): Уникальный идентификатор заказа (первичный ключ).
- `warehouse_id` (INTEGER): Идентификатор склада, на котором размещен заказ.
- `user_id` (INTEGER): Идентификатор пользователя, сделавшего заказ.
- `date` (TIMESTAMP): Дата и время оформления заказа.
- `paid_amount` (DECIMAL(10, 2)): Сумма, уплаченная за заказ.
- `quantity` (DECIMAL(10, 2)): Количество продуктов в заказе.
 
4. Таблица `order_line_mag`:
- `order_id` (INTEGER): Идентификатор заказа, к которому относится строка заказа (внешний ключ).
- `date` (TIMESTAMP): Дата и время, когда была добавлена строка заказа.
- `warehouse_id` (INTEGER): Идентификатор склада, на котором размещен продукт из строки заказа (внешний ключ).
- `product_id` (INTEGER): Идентификатор продукта из строки заказа (внешний ключ).
- `price` (DECIMAL(10, 2)): Цена продукта в строке заказа.
- `regular_price` (DECIMAL(10, 2)): Обычная цена продукта (без скидок) в строке заказа.
- `cost_price` (DECIMAL(10, 2)): Закупочная цена продукта в строке заказа.
- `quantity` (DECIMAL(10, 2)): Количество продуктов в строке заказа.
- `paid_amount` (DECIMAL(10, 2)): Сумма, уплаченная за продукты в строке заказа.
- 
5. Таблица `lost_mag`:
- `date` (TIMESTAMP): Дата и время, когда был утерян продукт (внешний ключ).
- `warehouse_id` (INTEGER): Идентификатор склада, на котором произошла утеря продукта (внешний ключ).
- `product_id` (INTEGER): Идентификатор продукта, который был утерян (внешний ключ).
- `item_id` (INTEGER): Идентификатор элемента продукта, который был утерян (внешний ключ).
- `quantity` (DECIMAL(10, 2)): Количество утерянных продуктов.
- `amount` (DECIMAL(10, 2)): Сумма утерянных продуктов (в валюте).

### Задача 1
Посчитайте количество работающих складов на текущую дату по каждому городу. 

Столбцы в результате: \
`city`, \
`count_warehouse`

### Решение:
```
SELECT 
    city, 
    count(warehouse_id) AS count_warehouse 
FROM 
    test.warehouses_mag 
WHERE 
    WHERE date_open < CURRENT_DATE AND (date_close > CURRENT_DATE OR date_close IS NULL) 
GROUP BY 
    city
```

### Задача 2
Посчитайте количество позиций, которые продавались в среднем на 1 складе. Выведите данные в разрезе городов. 

Столбцы в результате: \
`city`, \
`count_warehouse`, \
`mean_quantity_per_warehouse`

### Решение:
```
SELECT 
    wh_info.city, 
    wh_info.count_warehouse, 
    wh_info.sum_quantity / wh_info.count_warehouse AS mean_quantity_per_warehouse 
FROM 
  (
    SELECT 
        wh.city, 
        COUNT(orders_mag.warehouse_id) AS count_warehouse, 
        SUM(orders_mag.quantity) AS sum_quantity 
    FROM 
        test.orders_mag 
    LEFT JOIN 
        test.warehouses_mag AS wh on orders_mag.warehouse_id = wh.warehouse_id 
    GROUP BY 
        wh.city
  ) AS wh_info
```

### Задача 3
Посчитайте количество заказов и количество клиентов за 2023 год в разрезе месяцев.

Столбцы в результате: \
`month`, \
`orders_cnt`, \
`user_cnt`

### Решение:
```
SELECT 
    EXTRACT(month from date) AS month,
    COUNT(order_id) AS orders_cnt,
    COUNT(DISTINCT user_id) AS user_cnt    
FROM 
    test.orders_mag
WHERE 
    EXTRACT(year from date) = '2023'
GROUP BY
    EXTRACT(month from date)
```

### Задача 4
Посчитайте средний заказ в рублях по каждому складу за за первую половину 2023 года (с 1 января по 1 июня). \
При этом выведите в алфавитном порядке наименования только тех складов, где средний заказ выше или равен среднему заказу по городу.

Столбцы в результате: \
`name`, \
`city`, \
`avg_per_wh`, \
`avg_per_city`

### Решение:
```
WITH avg_wh AS (
    SELECT 
        warehouse_id, AVG(paid_amount) AS avg_per_wh
    FROM 
        test.orders_mag
    WHERE 
        date BETWEEN '2023-01-01' AND '2023-06-01'
    GROUP BY 
        warehouse_id
),
avg_city AS (
    SELECT 
        wh.city, 
        AVG(orders.paid_amount) AS avg_per_city
    FROM 
        test.orders_mag AS orders
    LEFT JOIN 
        test.warehouses_mag AS wh ON orders.warehouse_id = wh.warehouse_id
    WHERE 
        orders.date BETWEEN '2023-01-01' AND '2023-06-01'
    GROUP BY 
        wh.city
)
SELECT 
    wh.name, 
    wh.city, 
    avg_wh.avg_per_wh, 
    avg_city.avg_per_city
FROM 
    test.warehouses_mag AS wh
LEFT JOIN 
    avg_wh ON wh.warehouse_id = avg_wh.warehouse_id
LEFT JOIN 
    avg_city ON wh.city = avg_city.city
WHERE 
    avg_wh.avg_per_wh >= avg_city.avg_per_city
ORDER BY 
    wh.name
```

### Задача 5
Рассчитайте процент потерь (от суммы продаж, учитывая все статьи) и долю потерь в общеуй сумме потерь по компании в целом за первую половину 2023 года (с 1 января 2023 года по 1 июня 2023 года) по каждой группе товаров 2 уровня.

Столбцы в результате: \
`group1`, \
`group2`, \
`perc_of_lost`, \
`perc`

### Решение:
```
WITH t1 AS (
    SELECT 
        p.group1, 
        p.group2, 
        SUM(ol.paid_amount) AS sum_paid_amount
    FROM 
        test.order_line_mag AS ol
    LEFT JOIN 
        test.product_mag AS p ON ol.product_id = p.product_id
    WHERE 
        ol.date BETWEEN '2023-01-01' AND '2023-06-01'
    GROUP BY
        p.group1, p.group2
),
t2 AS (
    SELECT 
        p.group2, 
        SUM(l.amount) AS sum_lost
    FROM 
        test.lost_mag AS l
    LEFT JOIN
        test.product_mag AS p ON l.product_id = p.product_id
    WHERE
        l.date BETWEEN '2023-01-01' AND '2023-06-01'
    GROUP BY
        p.group2
)
SELECT 
    t1.group1, 
    t1.group2, 
    t2.sum_lost / t1.sum_paid_amount AS perc_of_lost, 
    t2.sum_lost / (SELECT SUM(sum_lost) FROM t2) AS perc
FROM 
    t1
LEFT JOIN 
    t2 ON t1.group2 = t2.group2
```